**1. MENGIMPORT LIBRARY PYTHON YANG DIBUTUHKAN**

In [ ]:
# Untuk pengolahan data
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans


# Untuk visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_palette('Set1')
sns.set()

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Untuk menghilangkan warnings saat plotting seaborn
import warnings
warnings.filterwarnings('ignore')

# Untuk mengupload file
import os

**2. MENYIAPKAN DATASET YANG AKAN DIGUNAKAN**

In [ ]:
from google.colab import files
upload = files.upload()

Saving Rekomendasi_Tourguide.csv to Rekomendasi_Tourguide (2).csv


In [ ]:
tour=pd.read_csv("Rekomendasi_Tourguide.csv",sep=';')
tour

,id_tourguide,name,experience_years,city,rating,price
0,1,Anisa Suryanto,5,Yogyakarta,4.8,500.0
1,2,Budi Santoso,3,Yogyakarta,4.7,550.0
2,3,Cindy Lim,4,Yogyakarta,4.5,500.0
3,4,Dharma Putra,4,Yogyakarta,4.7,550.0
4,5,Elisa Tan,6,Surabaya,4.6,700.0
...,...,...,...,...,...,...
147,148,Yudi Wijaya,5,Pasuruan,4.7,750.0
148,149,Siti Rahayu,7,Probolinggo,4.5,550.0
149,150,Rudi Wijaya,4,Probolinggo,4.8,600.0
150,151,Maya Santoso,6,Probolinggo,4.7,700.0


**3. DATA UNDERSTANDING**

In [ ]:
tour.head(5)

,id_tourguide,name,experience_years,city,rating,price
0,1,Anisa Suryanto,5,Yogyakarta,4.8,500.0
1,2,Budi Santoso,3,Yogyakarta,4.7,550.0
2,3,Cindy Lim,4,Yogyakarta,4.5,500.0
3,4,Dharma Putra,4,Yogyakarta,4.7,550.0
4,5,Elisa Tan,6,Surabaya,4.6,700.0


In [ ]:
tour.shape

(152, 6)

In [ ]:
tour.isnull().sum()

id_tourguide        0
name                0
experience_years    0
city                0
rating              0
price               0
dtype: int64

In [ ]:
tour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_tourguide      152 non-null    int64  
 1   name              152 non-null    object 
 2   experience_years  152 non-null    int64  
 3   city              152 non-null    object 
 4   rating            152 non-null    float64
 5   price             152 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 7.2+ KB


In [ ]:
tour.describe

<bound method NDFrame.describe of      id_tourguide            name  experience_years         city  rating  \
0               1  Anisa Suryanto                 5   Yogyakarta     4.8   
1               2    Budi Santoso                 3   Yogyakarta     4.7   
2               3       Cindy Lim                 4   Yogyakarta     4.5   
3               4    Dharma Putra                 4   Yogyakarta     4.7   
4               5       Elisa Tan                 6     Surabaya     4.6   
..            ...             ...               ...          ...     ...   
147           148     Yudi Wijaya                 5     Pasuruan     4.7   
148           149     Siti Rahayu                 7  Probolinggo     4.5   
149           150     Rudi Wijaya                 4  Probolinggo     4.8   
150           151    Maya Santoso                 6  Probolinggo     4.7   
151           152    Dina Nugroho                 3  Probolinggo     4.2   

     price  
0    500.0  
1    550.0  
2    500.0  
3

In [ ]:
# Konversi kolom 'price' ke tipe data numerik
tour['price'] = pd.to_numeric(tour['price'], errors='coerce')
# Menambahkan dua digit nol di depan kolom 'price'
tour['price'] = tour['price'].apply(lambda x: '{:.3f}'.format(x))
print(tour)

     id_tourguide            name  experience_years         city  rating  \
0               1  Anisa Suryanto                 5   Yogyakarta     4.8   
1               2    Budi Santoso                 3   Yogyakarta     4.7   
2               3       Cindy Lim                 4   Yogyakarta     4.5   
3               4    Dharma Putra                 4   Yogyakarta     4.7   
4               5       Elisa Tan                 6     Surabaya     4.6   
..            ...             ...               ...          ...     ...   
147           148     Yudi Wijaya                 5     Pasuruan     4.7   
148           149     Siti Rahayu                 7  Probolinggo     4.5   
149           150     Rudi Wijaya                 4  Probolinggo     4.8   
150           151    Maya Santoso                 6  Probolinggo     4.7   
151           152    Dina Nugroho                 3  Probolinggo     4.2   

       price  
0    500.000  
1    550.000  
2    500.000  
3    550.000  
4    700.000

**4. CLUSTERING BERDASARKAN CITY**

In [ ]:
# Pilih fitur yang ingin digunakan
features = ['experience_years', 'price']

# Lakukan clustering untuk setiap kota
for city in cities:
    city_data = tour[tour['city'] == city]

    # Hapus kolom 'name' dan 'city' sebelum clustering
    city_data = city_data.drop(['city'], axis=1)

    # Konversi kolom 'rating' menjadi numerik jika belum
    city_data['rating'] = pd.to_numeric(city_data['rating'], errors='coerce')

    # Hapus baris yang memiliki nilai NaN
    city_data = city_data.dropna(subset=features + ['rating'])

    # Inisialisasi model K-Means
    kmeans = KMeans(n_clusters=3)  # Ganti jumlah cluster sesuai kebutuhan

    # Latih model
    kmeans.fit(city_data[features])

    # Tampilkan hasil clustering
    print(f'Location Tourguide in {city}:')
    print(city_data[['name', 'experience_years', 'rating', 'price']])
    print('\n')


Location Tourguide in Yogyakarta:
             name  experience_years  rating    price
0  Anisa Suryanto                 5     4.8  500.000
1    Budi Santoso                 3     4.7  550.000
2       Cindy Lim                 4     4.5  500.000
3    Dharma Putra                 4     4.7  550.000


Location Tourguide in Surabaya:
            name  experience_years  rating    price
4      Elisa Tan                 6     4.6  700.000
5  Faisal Rahman                 5     4.2  600.000
6    Gita Wijaya                 4     4.5  650.000
7  Hanif Pratama                 6     4.8  700.000


Location Tourguide in Jakarta Barat:
            name  experience_years  rating    price
8   Indah Kusuma                 7     4.8  800.000
9    Joko Susilo                 4     4.4  700.000
10  Karina Putri                 6     4.9  900.000
11  Lukman Harun                 5     4.7  750.000


Location Tourguide in Bandung:
              name  experience_years  rating    price
12    Dian Pratiwi   